# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

- 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies

We'll first install all our required libraries.

> NOTE: If you're running this locally - please skip this step.

In [1]:
#!pip install -qU langchain langchain_openai langchain-community langgraph arxiv

## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

In [4]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE6 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Tavily Search Results](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/tools/tavily_search/tool.py)
- [Arxiv](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/arxiv)

#### 🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tavily_tool = TavilySearchResults(max_results=5)

tool_belt = [
    tavily_tool,
    ArxivQueryRun(),
]

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [7]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:

How does the model determine which tool to use?

### 😎 ANSWER #1: 
The model itself choose the right tool.
Thanks to its training and reasoning capabilities, the model determines which tool is best suited to answer a specific query.

## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [8]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [9]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [10]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [11]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [12]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [13]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [14]:
compiled_graph = uncompiled_graph.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

If not, how could we impose a limit to the number of cycles?

😎 #### ANSWER #2

1. No, by default there is no specific limit to the number of cycles we can perform in a graph. The cycle will continue as long as the condition defined in the conditional edges is verified.

2. To impose a limit on the number of cycles, we can modify our state and conditional function to track the number of iterations. 
First we add a counter to the state definiton class. Then we check this counter in the conditional function. If the maximum number of iterations has been reached : END.

## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [15]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CQJpcsGQhKl31uOlIcYCIN99', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 162, 'total_tokens': 189, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5377a239-354d-40f2-81de-b9f425f0899b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current captain of the Winnipeg Jets 2023'}, 'id': 'call_CQJpcsGQhKl31uOlIcYCIN99', 'type': 'tool_call'}], usage_metadata={'input_tokens': 162, 'output_t

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [16]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using Tavily!")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hDytmsI2EXlrwKIwmAZNcQtx', 'function': {'arguments': '{"query":"QLoRA"}', 'name': 'arxiv'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 178, 'total_tokens': 195, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1d27f908-64be-404f-be01-327178e0ddae-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'QLoRA'}, 'id': 'call_hDytmsI2EXlrwKIwmAZNcQtx', 'type': 'tool_call'}], usage_metadata={'input_tokens': 178, 'output_tokens': 17, 'total_tokens': 195, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'a

#### 🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

### 😎 ANSWER #2

Here are the steps the agent followed to arrive at the correct answer:

1. **Understanding the Initial Query**
- The agent received the user query: "Search Arxiv for the QLoRA paper, then search each of the authors to find their latest Tweet using Tavily!"
- The "agent" node processed this query and decided to use the "arxiv" tool to find the original paper.

2. **Searching for the QLoRA paper on ArXiv**
- The agent called the "arxiv" tool with the query "QLoRA" to find the original paper.
- The "action" node executed this tool call and returned the results.
- The agent received information about the paper "QLoRA: Efficient Finetuning of Quantized LLMs" as well as the authors' names: Tim Dettmers, Artidoro Pagnoni, Ari Holtzman, and Luke Zettlemoyer.

3. **Planning Author Searches**
- After identifying the authors, the agent decided to search for each author's latest tweets.
- It formulated four separate queries using the Tavily tool.

4. **Running Searches for Each Author**
- The agent sent four simultaneous queries using the "tavily_search_results_json" tool:
- "Tim Dettmers latest tweet"
- "Artidoro Pagnoni latest tweet"
- "Ari Holtzman latest tweet"
- "Luke Zettlemoyer latest tweet"
- The "action" node executed these queries and returned the results for each author.

5. **Summarizing Results**
- The agent analyzed the search results for each author.
- It extracted the most recent tweets from each author.
- It structured this information into a coherent and organized response.

6. **Presentation of Results**
- The agent presented each author's latest tweets in a readable format.
- For each author, it included a link to their tweet and directly quoted the content.
- It concluded with a brief note contextualizing this information.

# 🤝 Breakout Room #2

## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [17]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain = convert_inputs | compiled_graph | parse_output

In [18]:
agent_chain.invoke({"question" : "What is RAG?"})

"RAG stands for Retrieval-Augmented Generation. It is a technique used in natural language processing (NLP) that combines retrieval-based methods with generative models to improve the quality and relevance of generated text. Here's how it generally works:\n\n1. **Retrieval**: The system first retrieves relevant documents or pieces of information from a large corpus or database. This step is crucial for grounding the generative model in factual and contextually relevant information.\n\n2. **Augmentation**: The retrieved information is then used to augment the input to a generative model. This can involve concatenating the retrieved text with the original input or using it to inform the model's understanding of the context.\n\n3. **Generation**: Finally, a generative model, such as a transformer-based language model, uses the augmented input to produce a response or generate text. The goal is to create output that is not only coherent and contextually appropriate but also factually accur

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]
```

#### 🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions.

In [19]:
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]

Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [20]:
from langsmith import Client

client = Client()

dataset_name = f"Retrieval Augmented Generation - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

{'example_ids': ['877c8a8e-3e86-48d5-9b27-d45a172f750a',
  '64ce748d-a755-40e4-b09f-69cee8d46195',
  'b635b37b-dcd1-4897-8e20-ea03013922b4',
  'b0b889c1-c101-4ec6-819a-798870795ef9',
  'f6efd379-c5fe-4c23-99d6-28aedb0f7dc3',
  '5334bdc9-46c0-4adc-94ac-e724809ed35e'],
 'count': 6}

#### ❓ Question #3:

How are the correct answers associated with the questions?

> NOTE: Feel free to indicate if this is problematic or not

#### 😎 ANSWER #3

The correct answers are associated with questions through parallel lists (questions and answers) where each index corresponds to the same question-answer pair. When creating examples in LangSmith, these are combined using list comprehension to form inputs (questions) and outputs (expected answers with "must_mention" criteria).
 
This approach could be problematic because it relies on index alignment between separate lists. There's no explicit mapping between questions and answers.
If the lists get out of sync, the wrong answers will be associated with questions.

### Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [21]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

#### 😎 ANSWER #4

Disadvantages of this method:
- It is case-sensitive
- It does not take context or semantics into account
- It is sensitive to synonyms or paraphrases
- It only compares vocabulary but does not evaluate the overall quality of the answer

Suggestions for improvements:
- Use a more sophisticated scoring method that takes context and semantics into account
- Add a score for the overall quality of the answer
- Add metadata for the phrases that were matched or missing

In [22]:
# improves the metric by taking context and semantics into accoun
def must_mention_improved(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    
    if not required:
        return EvaluationResult(key="must_mention", score=1.0)
    
    # lower case the prediction
    prediction_lower = prediction.lower()
    matches = [phrase.lower() in prediction_lower for phrase in required]
    
    # Partial score
    score = sum(matches) / len(required) if required else 1.0
    
    # Add metadata for the phrases that were matched or missing
    missing_phrases = [required[i] for i, match in enumerate(matches) if not match]
    
    return EvaluationResult(
        key="must_mention_improved", 
        score=score,
        metadata={
            "matched_phrases": sum(matches),
            "total_required": len(required),
            "missing_phrases": missing_phrases
        }
    )

Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [23]:
experiment_results = client.evaluate(
    agent_chain,
    data=dataset_name,
    evaluators=[must_mention],
    experiment_prefix=f"RAG Pipeline - Evaluation - {uuid4().hex[0:4]}",
    metadata={"version": "1.0.0"},
)

View the evaluation results for experiment: 'RAG Pipeline - Evaluation - 63d1-c2eff263' at:
https://smith.langchain.com/o/0f3983de-eb9f-4417-bd1b-e1b02dc03750/datasets/55c149a4-9483-4591-9458-99ff736a1a5c/compare?selectedSessions=da45384d-3600-4757-ba15-57ee066b9246




0it [00:00, ?it/s]

In [24]:
experiment_results

,inputs.question,outputs.output,error,reference.must_mention,feedback.must_mention,execution_time,example_id,id
0,What optimizer is used in QLoRA?,"QLoRA uses ""paged optimizers"" to manage memory...",None,"[paged, optimizer]",True,5.549805,5334bdc9-46c0-4adc-94ac-e724809ed35e,853bc73f-2d57-4e3c-88de-d2462a5fd6fc
1,What is a Retrieval Augmented Generation system?,A Retrieval Augmented Generation (RAG) system ...,None,"[ground, context]",True,5.613846,64ce748d-a755-40e4-b09f-69cee8d46195,c41001cd-af62-44a6-bca6-a476938c8719
2,What data type was created in the QLoRA paper?,"In the QLoRA paper, a new data type called ""4-...",None,"[NF4, NormalFloat]",True,10.789043,877c8a8e-3e86-48d5-9b27-d45a172f750a,03da3f85-0dc7-4a78-ba6e-5975aea45372
3,What is the most popular deep learning framework?,"In 2023, the most popular deep learning framew...",None,"[PyTorch, TensorFlow]",True,6.997947,b0b889c1-c101-4ec6-819a-798870795ef9,098654cc-306a-42dd-b005-5c1c5748c019
4,Who authored the QLoRA paper?,"The QLoRA paper titled ""Accurate LoRA-Finetuni...",None,"[Tim, Dettmers]",False,13.065288,b635b37b-dcd1-4897-8e20-ea03013922b4,22256c10-85ca-438b-b1ee-d23e6fbf6315
5,What significant improvements does the LoRA sy...,The LoRA (Low-Rank Adaptation) system has made...,None,"[reduce, parameters]",False,12.777440,f6efd379-c5fe-4c23-99d6-28aedb0f7dc3,1b33bea2-a2e6-4525-b546-21ca35392da8


## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [25]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

#### 🏗️ Activity #5:

Please write markdown for the following cells to explain what each is doing.

#### This cell initializes our LangGraph by creating a StateGraph with the AgentState type. 
It adds two nodes to the graph:
1. "agent" node -> responsible for calling the language model
2. "action" node -> responsible for executing tools when the agent decides to use them
This forms the basic structure of our agent workflow.

In [26]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

#### This cell defines the entry point for our LangGraph workflow. 

By setting "agent" as the entry point, we explicitly say that the execution of our graph begin with the agent node, which is responsible for calling the language model to process the initial user input.

In [27]:
graph_with_helpfulness_check.set_entry_point("agent")

#### This cell defines the conditional edges in our graph :

1. From "agent" node: 
If the response of the agent contains tool calls, we route to the "action" node to execute those tools.
Otherwise, we check if the response is helpful by quering the model with an appropriate prompt

2. From "action" node:
We always return to the "agent" node after executing a tool, so the agent can process the tool output

So, we create a dynamic workflow where the agent can either use tools or provide direct responses, with a helpfulness check to survey the quality of the responses.

In [28]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "end"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4")

  helpfulness_chain = prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

#### 🏗️ Activity #4:

Please write what is happening in our `tool_call_or_helpful` function!

The `tool_call_or_helpful` function serves as a decision-making node in our LangGraph. It analyzes the current state and determines the next step in the workflow by:
- Checking if the last message contains tool calls - if so, it routes to the "action" node
- If the conversation exceeds 10 messages, it terminates the workflow
- Otherwise, it evaluates the helpfulness of the response by comparing the initial query with the final response using GPT-4
- Based on the helpfulness evaluation (Y/N), it either ends the conversation (if helpful) or continues the agent interaction (if not helpful)

#### This cell adds conditional edges to the graph:
- Starting from the "agent" node
- Using the "tool_call_or_helpful" function to determine the direction :
    * "continue": returns to the "agent" node to continue the conversation
    * "action": goes to the "action" node to execute a tool
    * "end": terminates the graph execution flow

In [29]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

####  This cell adds a direct edge from the "action" node back to the "agent" node in our graph.

In [30]:
graph_with_helpfulness_check.add_edge("action", "agent")

This cell compiles the graph so to verify and finalize the graph structure.
After that our graph is ready to use.

In [31]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

#### This cell creates an input with a question about LoRA, Tim Dettmers, and the Attention mechanism .

Then it streams the response asynchronously from our agent graph.
For each chunk received, it prints the node name and the messages content.
So we can see the step-by-step processing through different nodes of our graph.

In [32]:
inputs = {"messages" : [HumanMessage(content="Related to machine learning, what is LoRA? Also, who is Tim Dettmers? Also, what is Attention?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CSTN2psRlEth9ywoZaej6Dln', 'function': {'arguments': '{"query": "LoRA machine learning"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_WKH0ZFdupldj9HYY1Zp17VZB', 'function': {'arguments': '{"query": "Tim Dettmers"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_XTssjDVZE0p3KcKGhDcpGPj2', 'function': {'arguments': '{"query": "Attention mechanism in machine learning"}', 'name': 'arxiv'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 177, 'total_tokens': 250, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'finish_reason': 'tool_calls', 'logprobs': Non

### Task 4: LangGraph for the "Patterns" of GenAI

Let's ask our system about the 4 patterns of Generative AI:

1. Prompt Engineering
2. RAG
3. Fine-tuning
4. Agents

In [33]:
patterns = ["prompt engineering", "RAG", "fine-tuning", "LLM-based agents"]

In [34]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

**Prompt Engineering: Definition and Overview**

Prompt engineering is the process of designing and refining input prompts to effectively guide the behavior of AI models, particularly generative AI models like ChatGPT or DALL-E. It involves crafting instructions or queries in natural language to elicit specific responses from AI systems. This practice is crucial for improving the accuracy and effectiveness of AI-generated outputs. Prompt engineering can involve specifying a style, choice of words, grammar, providing relevant context, or describing a character for the AI to mimic. It is a technique used by AI engineers to refine large language models (LLMs) with specific or recommended prompts.

**History and Emergence of Prompt Engineering**

Prompt engineering has its roots in the early days of natural language processing (NLP) and has evolved alongside the development of AI systems. The concept gained significant attention with the release of GPT-3 by OpenAI in 2020, which marked a m

In [35]:
for pattern in patterns:    
# Créer la question pour le modèle
    what_is_string = f"What is {pattern} and when did it break onto the scene??"
    
    # Préparer l'entrée pour notre agent
    inputs = {"messages": [HumanMessage(content=what_is_string)]}
    
    # Invoquer l'agent et récupérer la réponse
    messages = agent_with_helpfulness_check.invoke(inputs)
    
    # Afficher le résultat avec un formatage amélioré et des retours chariots
    print("\n" + "="*80)
    print(f"📌 PATTERN: {pattern.upper()}")
    print("="*80 + "\n")
    
    # Formater la réponse avec des sauts de ligne préservés
    response_content = messages["messages"][-1].content
    print(response_content)
    
    print("\n" + "="*80 + "\n\n")


📌 PATTERN: PROMPT ENGINEERING

**Prompt Engineering: Definition and Overview**

Prompt engineering is the process of designing and refining input prompts to effectively guide the behavior of AI models, particularly generative AI models like ChatGPT or DALL-E. It involves crafting instructions or queries in natural language to elicit specific responses from AI models. This practice is crucial for improving the accuracy and effectiveness of AI-generated outputs. Prompt engineering can involve specifying a style, choice of words, grammar, providing relevant context, or describing a character for the AI to mimic. It is a technique used by AI engineers to refine large language models (LLMs) with specific or recommended prompts.

**History and Emergence of Prompt Engineering**

Prompt engineering has its roots in the early days of natural language processing (NLP) and has evolved alongside the development of AI systems. The concept gained significant attention with the release of GPT-3 by O

In [36]:
# Let's create a new chain for our agent with helpfulness check
def convert_inputs_for_helpfulness(input_object):
    return {"messages": [HumanMessage(content=input_object["question"])]}

def parse_output_for_helpfulness(input_state):
    return input_state["messages"][-1].content

# Creating the chain for evaluation
agent_with_helpfulness_chain = convert_inputs_for_helpfulness | agent_with_helpfulness_check | parse_output_for_helpfulness


# Creating a new experiment identifier
experiment_name = f"RAG Pipeline with helpfulness check - {uuid4().hex[0:4]}"

# Launching the evaluation with the same dataset and evaluator
experiment_results_helpfulness = client.evaluate(
    agent_with_helpfulness_chain,
    data=dataset_name,  # Using the same dataset as before
    evaluators=[must_mention_improved],  # Using the improved evaluator
    experiment_prefix=experiment_name,
    metadata={"version": "1.1.0", "description": "Agent with helpfulness check"}
)


View the evaluation results for experiment: 'RAG Pipeline with helpfulness check - 05f2-6b573b44' at:
https://smith.langchain.com/o/0f3983de-eb9f-4417-bd1b-e1b02dc03750/datasets/55c149a4-9483-4591-9458-99ff736a1a5c/compare?selectedSessions=adfbcf6a-d177-4802-a57a-5dbc188296de




0it [00:00, ?it/s]

In [37]:
experiment_results

,inputs.question,outputs.output,error,reference.must_mention,feedback.must_mention,execution_time,example_id,id
0,What optimizer is used in QLoRA?,"QLoRA uses ""paged optimizers"" to manage memory...",None,"[paged, optimizer]",True,5.549805,5334bdc9-46c0-4adc-94ac-e724809ed35e,853bc73f-2d57-4e3c-88de-d2462a5fd6fc
1,What is a Retrieval Augmented Generation system?,A Retrieval Augmented Generation (RAG) system ...,None,"[ground, context]",True,5.613846,64ce748d-a755-40e4-b09f-69cee8d46195,c41001cd-af62-44a6-bca6-a476938c8719
2,What data type was created in the QLoRA paper?,"In the QLoRA paper, a new data type called ""4-...",None,"[NF4, NormalFloat]",True,10.789043,877c8a8e-3e86-48d5-9b27-d45a172f750a,03da3f85-0dc7-4a78-ba6e-5975aea45372
3,What is the most popular deep learning framework?,"In 2023, the most popular deep learning framew...",None,"[PyTorch, TensorFlow]",True,6.997947,b0b889c1-c101-4ec6-819a-798870795ef9,098654cc-306a-42dd-b005-5c1c5748c019
4,Who authored the QLoRA paper?,"The QLoRA paper titled ""Accurate LoRA-Finetuni...",None,"[Tim, Dettmers]",False,13.065288,b635b37b-dcd1-4897-8e20-ea03013922b4,22256c10-85ca-438b-b1ee-d23e6fbf6315
5,What significant improvements does the LoRA sy...,The LoRA (Low-Rank Adaptation) system has made...,None,"[reduce, parameters]",False,12.777440,f6efd379-c5fe-4c23-99d6-28aedb0f7dc3,1b33bea2-a2e6-4525-b546-21ca35392da8


In [38]:
experiment_results_helpfulness


,inputs.question,outputs.output,error,reference.must_mention,feedback.must_mention_improved,execution_time,example_id,id
0,What optimizer is used in QLoRA?,"QLoRA uses ""paged optimizers"" to manage memory...",None,"[paged, optimizer]",1.0,6.368424,5334bdc9-46c0-4adc-94ac-e724809ed35e,dfd20b3f-1ccf-4a7e-a9aa-a974a30a896e
1,What is a Retrieval Augmented Generation system?,A Retrieval Augmented Generation (RAG) system ...,None,"[ground, context]",1.0,7.297039,64ce748d-a755-40e4-b09f-69cee8d46195,c2d537ac-8d0b-426c-b596-41af755309d9
2,What data type was created in the QLoRA paper?,"In the QLoRA paper, a new data type called ""4-...",None,"[NF4, NormalFloat]",1.0,8.510165,877c8a8e-3e86-48d5-9b27-d45a172f750a,af76536f-5803-43e6-a567-59668d558810
3,What is the most popular deep learning framework?,"In 2023, the most popular deep learning framew...",None,"[PyTorch, TensorFlow]",1.0,8.433764,b0b889c1-c101-4ec6-819a-798870795ef9,9e86ac3b-255d-4ff9-9356-fcf6439a62b1
4,Who authored the QLoRA paper?,"The QLoRA paper titled ""Accurate LoRA-Finetuni...",None,"[Tim, Dettmers]",0.0,7.831095,b635b37b-dcd1-4897-8e20-ea03013922b4,05f0a4c2-efab-42ce-89f5-22f19f8e4a3e
5,What significant improvements does the LoRA sy...,The LoRA (Low-Rank Adaptation) system has made...,None,"[reduce, parameters]",0.0,16.820076,f6efd379-c5fe-4c23-99d6-28aedb0f7dc3,580756c1-b6db-4bef-ad4b-8ae4d6f8da4e
